In [1]:
import os
from dotenv import load_dotenv
from tabula import read_pdf
import pandas as pd

In [2]:
pdf_path = "/Users/home/Downloads/Contract Notes/"
pdf_paths = [os.path.join(pdf_path, f) for f in os.listdir(pdf_path) if f.endswith('.pdf')]

In [3]:
def get_data_from(pdf_path: str) -> pd.DataFrame:
    load_dotenv()
    password = os.getenv('PDF_PASSWORD')

    table = read_pdf(pdf_path, pages=1, password=password)[0]
    table = table.drop(table.tail(1).index)
    table.columns = table.iloc[0]
    table = table.iloc[1:]
    table['txn_date'] = pd.to_datetime(pdf_path.strip('.pdf')[-11:], format='%d-%b-%Y').strftime('%Y-%m-%d')

    return table

In [4]:
for pdf_path in pdf_paths:
    df = get_data_from(pdf_path)
    if pdf_path == pdf_paths[0]:
        table = df.copy()
    else:
        table = pd.concat([table, df], ignore_index=True)

In [5]:
table = table.rename(columns={
    'Segment': 'exchange',
    'Scrip Name': 'scrip',
    'Buy Qty': 'buy_qty',
    'Avg. Buy Price (in Rs.)': 'buy_price',
    'Sell Qty': 'sell_qty',
    'Avg. Sell Price (in Rs.)': 'sell_price',
    'Brokerage (in Rs.)': 'brokerage',
    'STT (in Rs.)': 'stt',
    'Other Charges (in Rs.)': 'other_charges',
    'Total (in Rs.)': 'total'
})

table['exchange'] = table['exchange'].str[-3:]
table = table[['txn_date'] + [col for col in table.columns if col != 'txn_date']]
table = table.sort_values(['txn_date', 'scrip']).reset_index(drop=True)

table.to_csv('stock_trades.csv', index=False)
table

,txn_date,exchange,scrip,buy_qty,buy_price,sell_qty,sell_price,brokerage,stt,other_charges,total
0,2024-05-22,NSE,AUBANK,10,618.80,0,0.00,15.00,5.76,3.94,-6212.70
1,2024-05-22,NSE,BERGEPAINT,15,492.70,0,0.00,15.00,7.39,3.99,-7416.88
2,2024-05-22,NSE,CEATLTD,4,2392.55,0,0.00,15.00,9.57,4.07,-9598.84
3,2024-05-22,NSE,CREDITACC,5,1412.35,0,0.00,15.00,7.06,3.98,-7087.79
4,2024-05-22,NSE,CRISIL,3,4448.25,0,0.00,15.00,13.34,5.22,-13378.31
...,...,...,...,...,...,...,...,...,...,...,...
147,2024-12-09,NSE,JBMA,4,1611.50,0,0.00,15.00,6.45,3.93,-6471.38
148,2024-12-09,NSE,SSDL,24,140.32,0,0.00,15.00,3.37,3.82,-3389.87
149,2024-12-09,NSE,WELCORP,0,0.00,10,801.55,15.00,7.95,2.99,7989.56
150,2024-12-16,NSE,MAXHEALTH,0,0.00,6,1182.65,15.00,7.00,2.96,7070.94
